In [18]:
!pip install pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
spark = SparkSession.builder.enableHiveSupport().getOrCreate()
from google.colab import drive
drive.mount("/content/drive",force_remount=True)

base_path = "/content/drive/MyDrive/tweet_vaccines/"
import pandas as pd
tweets = pd.read_csv(base_path + "df_tweets_2021-07-15.csv", sep="|")
tweets.head(5)

Mounted at /content/drive


,hashtag,timestamp,tweet_text,user_name,language
0,#vaccines,2021-07-14 23:59:48,When I hear #Trudeau &amp; CO. self praise abo...,vitevoila,en
1,#vaccines,2021-07-14 23:58:48,Compliance to adjust oversight practices with ...,_UpFRONT_,en
2,#vaccines,2021-07-14 23:58:02,#COVID19 #vaccines risks/benefits:,bok_bok_ba_GOK,en
3,#vaccines,2021-07-14 23:55:57,Vietnam's health ministry has confirmed that U...,Madhuryavalli1,en
4,#vaccines,2021-07-14 23:53:32,"Why are so-called ""pro-choice liberals"" NOT pr...",PRAISETRIUNEGOD,en


In [19]:
import nltk
nltk.download('punkt')
tweets["tweet_text"] = tweets["tweet_text"].astype(str).apply(nltk.word_tokenize)
tweets.head(5)
print (tweets.shape)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
(15271, 5)


In [33]:
df = pd.DataFrame() 
lista = [] 
listb = []
for i in range (0,15271):
  for x in tweets.loc[i,"tweet_text"]:
    lista.append(i)
    listb.append(x)
d = {'basket':lista,'item':listb}
dframe = pd.DataFrame(d)  
dframe.head(20)

,basket,item
0,0,When
1,0,I
2,0,hear
3,0,#
4,0,Trudeau
5,0,&
6,0,amp
7,0,;
8,0,CO.
9,0,self


In [34]:
!pip install pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [35]:
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

In [36]:
df_sp = spark.createDataFrame(dframe)

In [39]:
df_sp.createOrReplaceTempView("df_sp")

In [41]:
df_sp.show(n = 30)

+------+--------+
|basket|    item|
+------+--------+
|     0|    When|
|     0|       I|
|     0|    hear|
|     0|       #|
|     0| Trudeau|
|     0|       &|
|     0|     amp|
|     0|       ;|
|     0|     CO.|
|     0|    self|
|     0|  praise|
|     0|   about|
|     0| vaccine|
|     0|  supply|
|     0|       &|
|     0|     amp|
|     0|       ;|
|     0|progress|
|     0|       ,|
|     0|       I|
|     0|     see|
|     0|      it|
|     0|      as|
|     0|       a|
|     0|   smack|
|     0|      in|
|     0|     the|
|     0|    face|
|     0|      to|
|     0|     our|
+------+--------+
only showing top 30 rows



In [43]:
from pyspark.sql.functions import collect_set

baskets = df_sp.groupBy('basket').agg(collect_set('item').alias('items'))
baskets.createOrReplaceTempView('baskets')
baskets.toPandas().head(40)

,basket,items
0,26,"[the, ?, been, depressed, associated, that, wi..."
1,29,"[Vaccines, the, education, ,, No, @, and, unde..."
2,474,"[,, may, CovidCoalition, storage, health, ones..."
3,964,"[the, out, JohnsonVariant, website, on, UK, Jo..."
4,1677,"[Thailand, market, William67335893, @, in]"
5,1697,"[again, population, -, The, COVID, And, surge,..."
6,1806,"[,, serious, Take, health, 'surge, News, delta..."
7,1950,"[,, Mary, Senior, transmissible, Coronavirus, ..."
8,2040,"[,, the, is, ?, ASEAN, UNinMyanmar, UNHumanRig..."
9,2214,"[,, lives, can, vaccinated, faculty, including..."


In [44]:
from pyspark.ml.fpm import FPGrowth

fpGrowth = FPGrowth(itemsCol="items", minSupport=0.001, minConfidence=0)
model = fpGrowth.fit(baskets)

In [ ]:
# Display frequent itemsets.
mostPopularItemInABasket = model.freqItemsets
mostPopularItemInABasket.createOrReplaceTempView("mostPopularItemInABasket")
mostPopularItemInABasket.show()

# Display generated association rules.
associationRules = model.associationRules
associationRules.createOrReplaceTempView("associationRules")
associationRules.show()

# transform examines the input items against all the association rules and summarize the consequents as prediction
associations = model.transform(baskets)
associations.show()

+--------------------+----+
|               items|freq|
+--------------------+----+
|              [2021]| 249|
|          [2021, ``]|  16|
|       [2021, ``, :]|  16|
|    [2021, ``, :, .]|  16|
| [2021, ``, :, ., #]|  16|
|    [2021, ``, :, #]|  16|
|       [2021, ``, .]|  16|
|    [2021, ``, ., #]|  16|
|       [2021, ``, #]|  16|
|           [2021, )]|  39|
|      [2021, ), the]|  21|
|   [2021, ), the, #]|  18|
|        [2021, ), ,]|  37|
|   [2021, ), ,, the]|  21|
|[2021, ), ,, the, #]|  18|
|     [2021, ), ,, .]|  20|
|  [2021, ), ,, ., #]|  20|
|     [2021, ), ,, #]|  34|
| [2021, ), vaccines]|  22|
|[2021, ), vaccine...|  20|
+--------------------+----+
only showing top 20 rows



In [ ]:
query = """select items, freq
           from mostPopularItemInABasket
           where size(items) > 2
           order by freq desc
           limit 20"""
spark.sql(query).show()

In [ ]:
query = """select antecedent as `antecedent (if)`, consequent as `consequent (then)`, confidence
           from associationRules
           order by confidence desc
           limit 20"""

spark.sql(query).show()